In [1]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.ai_agent_ws/src/learn_llm/learn_rag_and_agent/../.env


In [2]:
import faiss
from langchain_openai import OpenAIEmbeddings
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory

# 임베딩 모델을 정의합니다.
embeddings_model = OpenAIEmbeddings()

embedding_size = 1536

index = faiss.IndexFlatL2(embedding_size)

vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

In [3]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

memory = VectorStoreRetrieverMemory(retriever=retriever)

# 임의의 대화를 저장합니다.
memory.save_context(
    inputs={
        "human": "안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다."
    },
    outputs={
        "ai": "안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다."
    },
)

memory.save_context(
    inputs={"human": "프로젝트에서 어떤 역할을 맡았나요?"},
    outputs={
        "ai": "제가 맡은 역할은 백엔드 개발자였습니다. 사용자 데이터 처리와 서버 로직 개발을 담당했으며, RESTful API를 구현하여 프론트엔드와의 통신을 담당했습니다. 또한, 데이터베이스 설계에도 참여했습니다."
    },
)

memory.save_context(
    inputs={
        "human": "팀 프로젝트에서 어려움을 겪었던 경험이 있다면 어떻게 해결했나요?"
    },
    outputs={
        "ai": "프로젝트 초기에 의사소통 문제로 몇 가지 어려움이 있었습니다. 이를 해결하기 위해 저희 팀은 정기적인 미팅을 갖고 각자의 진행 상황을 공유했습니다. 또한, 문제가 발생했을 때는 적극적으로 의견을 나누고, 합리적인 해결책을 찾기 위해 노력했습니다."
    },
)

memory.save_context(
    inputs={"human": "개발자로서 자신의 강점은 무엇이라고 생각하나요?"},
    outputs={
        "ai": "제 강점은 빠른 학습 능력과 문제 해결 능력입니다. 새로운 기술이나 도구를 빠르게 습득할 수 있으며, 복잡한 문제에 직면했을 때 창의적인 해결책을 제시할 수 있습니다. 또한, 팀워크를 중시하며 동료들과 협력하는 것을 중요하게 생각합니다."
    },
)

/tmp/ipykernel_16238/3099840607.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever)


In [4]:
# 저장할 로컬 경로 지정
# 이 폴더 안에 FAISS 인덱스 파일(index.faiss)과 문서 파일(index.pkl)이 저장됩니다.
PERSIST_DIR = "./conversation/faiss_memory" 

# vectorstore 객체를 사용하여 로컬에 저장
vectorstore.save_local(
    folder_path=PERSIST_DIR, 
    index_name="interview_history" # 저장될 파일의 이름 지정 (예: interview_history.faiss, interview_history.pkl)
) 

print(f"✅ 메모리가 로컬 폴더: {PERSIST_DIR} 에 성공적으로 저장되었습니다.")

✅ 메모리가 로컬 폴더: ./conversation/faiss_memory 에 성공적으로 저장되었습니다.


In [5]:
PERSIST_DIR = "./conversation/faiss_memory" 

# 1. 저장된 FAISS 객체를 로컬에서 불러옵니다.
#   - 이때 임베딩 모델(embeddings_model)은 다시 지정해 주어야 합니다.
loaded_vectorstore = FAISS.load_local(
    folder_path=PERSIST_DIR, 
    embeddings=embeddings_model, 
    allow_dangerous_deserialization=True,
    index_name="interview_history" # 저장 시 사용한 이름과 동일해야 합니다.
)

In [6]:
# 2. 로드된 vectorstore를 사용하여 retriever와 memory를 다시 설정합니다.
loaded_retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": 1})

loaded_memory = VectorStoreRetrieverMemory(retriever=loaded_retriever)

print("✅ 저장된 메모리를 성공적으로 불러왔습니다.")

✅ 저장된 메모리를 성공적으로 불러왔습니다.


In [7]:
# 3. 이제 loaded_memory를 사용하여 검색을 수행할 수 있습니다.
print(loaded_memory.load_memory_variables({"human": "면접자 전공은 무엇인가요?"})["history"])

human: 안녕하세요, 오늘 면접에 참석해주셔서 감사합니다. 자기소개 부탁드립니다.
ai: 안녕하세요. 저는 컴퓨터 과학을 전공한 신입 개발자입니다. 대학에서는 주로 자바와 파이썬을 사용했으며, 최근에는 웹 개발 프로젝트에 참여하여 실제 사용자를 위한 서비스를 개발하는 경험을 했습니다.
